# Exporting ImageNet Inception

In this notebook, we'll show how to export the [pre-trained Imagenet Inception model](https://tfhub.dev/google/imagenet/inception_v3/classification/3) for serving.

First, we'll install the required packages:

In [ ]:
!pip install tensorflow-hub==0.8.*

Define the model output path. The shared volume for model storage is mounted at `/models`.

In [ ]:
export_dir = "/models/inception"

We'll download the model from TensorFlow Hub and export it for serving:

In [ ]:
import time
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.python.saved_model.signature_def_utils_impl import predict_signature_def

# Inception v3 image input size
HEIGHT=299
WIDTH=299
CHANNELS=3

def prepare_image(image_str_tensor):
    """
    This implements the standard preprocessing that needs to be applied to the
    image tensors before passing them to the model. This is used for all input
    types.
    """
    image = tf.image.decode_jpeg(image_str_tensor, channels=CHANNELS)
    image = tf.expand_dims(image, 0)
    image = tf.compat.v1.image.resize_bilinear(image, [HEIGHT, WIDTH], align_corners=False)
    image = tf.squeeze(image, axis=[0])
    image = tf.cast(image, dtype=tf.uint8)
    return image


versioned_export_dir = export_dir + "/" + str(time.time()).split('.')[0]
builder = tf.compat.v1.saved_model.builder.SavedModelBuilder(versioned_export_dir)

with tf.compat.v1.Session(graph=tf.Graph()) as sess:
    print ("# Downloading")
    module = hub.Module("https://tfhub.dev/google/imagenet/inception_v3/classification/3")

    input_ph = tf.compat.v1.placeholder(tf.string, shape=[None])
    
    # https://github.com/tensorflow/tensorflow/issues/28007
    with tf.device("/cpu:0"):
        images_tensor = tf.map_fn(prepare_image, input_ph, back_prop=False, dtype=tf.uint8)
        
    images_tensor = tf.image.convert_image_dtype(images_tensor, dtype=input_params["images"].dtype)
    
    sess.run([tf.compat.v1.global_variables_initializer(), tf.compat.v1.tables_initializer()])

    classes = module(images_tensor)
    signature = predict_signature_def(inputs={"image_bytes": input_ph}, outputs={"classes": classes})

    builder.add_meta_graph_and_variables(
        sess, ["serve"], signature_def_map={"serving_default": signature}, strip_default_attrs=True
    )

print ("# Exporting")
builder.save()


Verify that the exported model now exists in the model volume

In [ ]:
! ls -lR /models/

That's it!

*This notebook is derived from original work by [Cortex](https://www.cortex.dev)*